In [5]:
import pandas as pd
import io
import mlflow

from domino.data_sources import DataSourceClient
from domino_data.datasets import DatasetClient


def clean_and_save_cc_file(raw_filename: str, clean_filename: str):
    # Load with force numeric, bad values → NaN
    object_store = DataSourceClient().get_datasource("credit_card_fraud_detection")
    
    # Stream to in-memory buffer
    buf = io.BytesIO()
    object_store.download_fileobj(raw_filename, buf)
    buf.seek(0)
    
    # Read into DataFrame in-memory
    df_raw = pd.read_csv(buf)
    
    print(df_raw.head())

    # Coerce all columns to numeric, invalid strings become NaN
    df_numeric = df_raw.apply(pd.to_numeric, errors='coerce')

    # Keep rows with no NaN anywhere
    mask = df_numeric.notna().all(axis=1)
    df_cleaned = df_numeric[mask]

    # Convert back to in-memory buffer
    clean_buf = io.BytesIO()
    df_cleaned.to_csv(clean_buf, index=False)
    clean_buf.seek(0)

    # Upload to Dataset
    dataset = DatasetClient().get_dataset(dataset_name)
    dataset.upload_fileobj(clean_filename, clean_buf)

    # df_cleaned.to_csv(clean_file, index=False)
    print(f"✅ Cleaned data saved to: {clean_filename}. Removed {len(df_raw) - len(df_cleaned)} rows")
    # TODO get this to save to a dataset

raw_filename = 'raw_cc_transaction_rolling_2d_window.csv'
clean_filename = '[PATH HERE]/cleaned_cc_transaction_rolling_2d_window.csv'

clean_and_save_cc_file(raw_filename, clean_filename)



/tmp/ipykernel_388/3550456014.py:19: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(buf)


  Time RelSpend-Z   dTime-Z GeoJump-Z Log-Amount POS-ModeRisk  MCC-Risk  \
0  0.0  -1.359807 -0.072781  2.536347   1.378155    -0.338321  0.462388   
1  0.0   1.191857  0.266151   0.16648   0.448154     0.060018 -0.082361   
2  1.0  -1.358354 -1.340163  1.773209    0.37978    -0.503198  1.800499   
3  1.0  -0.966272 -0.185226  1.792993  -0.863291    -0.010309  1.247203   
4  2.0  -1.158233  0.877737  1.548718   0.403034    -0.407193  0.095921   

  Device-Risk  Lon-Jump  Lat-Jump  ...  TxnSim-3 RollSpend-24h RollSpend-7d  \
0    0.239599  0.098698  0.363787  ... -0.018307      0.277838    -0.110474   
1   -0.078803  0.085102 -0.255425  ... -0.225775     -0.638672     0.101288   
2    0.791461  0.247676 -1.514654  ...  0.247998      0.771679     0.909412   
3    0.237609  0.377436 -1.387024  ...   -0.1083      0.005274    -0.190321   
4    0.592941 -0.270533  0.817739  ... -0.009431      0.798278    -0.137458   

  RollSpend-30d  SoftBL-1  SoftBL-2 Manual-Review Chargeback-Flag  Amount 

NameError: name 'dataset_name' is not defined